# Mobility Meeting Scheduler

#### Constants

In [177]:
SOLVER_TIME_LIMIT = 2000

#### Import Data

In [178]:
import json

# Import students information
file = open('../../data/students.json')

input = json.load(file)

DESTINATIONS = input['destinations']
STUDENTS_INFO = input['students']
N_STUDENTS = len(STUDENTS_INFO)

STUDENTS_ORIGINS = [DESTINATIONS.index(x) for x in [student["city"] for student in STUDENTS_INFO]]

MINIMUM_USEFUL_TIME = input['minimumTime']

print("Imported", N_STUDENTS, "students,", len(DESTINATIONS), "destinations and the minimum time.")


# Import flights
file = open('../../data/flights.json')

FLIGHTS = json.load(file)

FLIGHTS_ORIGINS = [DESTINATIONS.index(x) for x in [flight["origin"] for flight in FLIGHTS]]
FLIGHTS_DESTINATIONS = [DESTINATIONS.index(x) for x in [flight["destination"] for flight in FLIGHTS]]
FLIGHTS_COSTS = [int(flight["price"]) for flight in FLIGHTS]

print("Imported", len(FLIGHTS), "flights!")

Imported 3 students, 3 destinations and the minimum time.
Imported 113 flights!


#### Model

In [179]:
from docplex.cp.model import CpoModel

model = CpoModel()

#### Variables

In [180]:
Destination = model.integer_var(0, len(DESTINATIONS) - 1, "Destination")

StudentsFlights = [model.integer_var_list(2, -1, len(FLIGHTS) - 1) for i in range(N_STUDENTS)]

#### Contraints

In [181]:
for i in range(N_STUDENTS):
    Outgoing, Incoming = StudentsFlights[i]
    StudentOrigin = model.element(STUDENTS_ORIGINS, i)

    # Origin of the flights
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ORIGINS, Outgoing) == StudentOrigin))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_ORIGINS, Incoming) == Destination))

    # Destination of the flights
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DESTINATIONS, Outgoing) == Destination))
    model.add(model.if_then(StudentOrigin != Destination, model.element(FLIGHTS_DESTINATIONS, Incoming) == StudentOrigin))


# Minimize total cost
TotalCost = model.sum([model.conditional(model.element(STUDENTS_ORIGINS, i) != Destination, model.element(FLIGHTS_COSTS, StudentsFlights[i][0]) + model.element(FLIGHTS_COSTS, StudentsFlights[i][1]), 0) for i in range(N_STUDENTS)])
model.add(model.minimize(TotalCost))

#### Solve

In [182]:
print("Solving model....")

solution = model.solve(TimeLimit = SOLVER_TIME_LIMIT)

Solving model....
 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 7 variables, 12 constraints
 ! TimeLimit            = 2000
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 42.5 (before), 42.5 (after)
 !  . Memory usage      : 332.0 kB (before), 332.0 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          7                 -
 + New bound is 0
                        0          7    1   F        -
 + New bound is 170
                        2          7    1   F     0 != Destination
 + New bound is 171
 *           383        9  0.05s        1      (gap is 55.35%)
 *           380       47  0.05s        1      (gap is 55.00%)
 *           378       72  0.05s        1      (gap is 54.76%)
             378     

#### Solution

In [183]:
if solution:
    print("Solution status: " + solution.get_solve_status())
    print("--> Destination: " + DESTINATIONS[solution[Destination]])
    print("--> Total Cost: " + str(solution.get_objective_values()[0]))
    print("--> Students:")

    for i in range(N_STUDENTS):
        Outgoing, Incoming = StudentsFlights[i]

        print("    --> Student " + str(i + 1) + " departing from " + STUDENTS_INFO[i]["city"] + ":")

        if STUDENTS_ORIGINS[i] == solution[Destination]:
            print("       - Student need not to take any flights!")
        else:
            print("       -", list(FLIGHTS[solution[Outgoing]].items())[2:])
            print("       -", list(FLIGHTS[solution[Incoming]].items())[2:])

else:
    print("No solution found")

Solution status: Optimal
--> Destination: budapest
--> Total Cost: 378
--> Students:
    --> Student 1 departing from budapest:
       - Student need not to take any flights!
    --> Student 2 departing from zagreb:
       - [('departure', '05/05/2022, 14:15:00 '), ('arrival', '05/05/2022, 19:40:00 '), ('duration', 325), ('price', '105'), ('stops', 1)]
       - [('departure', '05/05/2022, 08:40:00 '), ('arrival', '05/05/2022, 16:55:00 '), ('duration', 495), ('price', '85'), ('stops', 1)]
    --> Student 3 departing from wien:
       - [('departure', '07/05/2022, 07:00:00 '), ('arrival', '07/05/2022, 14:55:00 '), ('duration', 475), ('price', '102'), ('stops', 1)]
       - [('departure', '06/05/2022, 09:15:00 '), ('arrival', '06/05/2022, 14:45:00 '), ('duration', 330), ('price', '86'), ('stops', 1)]
